# 结构化数据类型选择


在执行程序时,如果内存中有大量的活动对象,那么就有可能出现内存问题.尤其在内存有限或者需要处理大规模数据时,我们更加需要对数据对象的内存精打细算.


总结下目前截至python 3.7为止python自带的可以用于描述结构化数据的类型,包括:

+ `dict`字典类型
+ `class`用常规的类定义,标准库`dataclasses`3.7版本加入,可以简化定义
+ `class with __slots__`用`__slots__`限定的类定义结构化数据
+ `collections.namedtuple` 使用具名元组描述结构化数据,也可以使用`typing.NamedTuple`为其中的字段提供类型申明

本文将一个三维点来作为例子,探讨下不同数据类型的内存使用量

## 字典类型

字典类型一定是最占内存的结构化数据类型,因为它是hashmap,必然会造成大量空间的浪费.

在python3.6开始dict使用的是有序键,因此结构更紧凑了,但这并不会改变它是hashmap这个事实.

In [1]:
import sys

In [2]:
ob = {"x":1,"y":2,"z":3}
print(sys.getsizeof(ob))

240


也就是每1000000个数据会占用240m的内存

## 常规定义的类实例

类实例会比字典更加紧凑,

这个类类实例的结构大致如下:


字段|大小(bit)
---|---
`PyGC_Head`|24
`PyObject_Head`|16
`__weakref__`|8
`__dict__`|8

在python3.3开始,所有类实例的字典键`__dict__`都会存储在共享空间,因此内存中的实例大小也就减小了,但注意,这里的`__weakref__`和`__dict__`是信息的引用,他们只是放在了别处并不是不产生内存占用.

In [6]:
class Point:
    def __init__(self,x,y,z):
        self.x = x
        self.y = y
        self.z = z

ob = Point(1,2,3)
print(sys.getsizeof(ob))
print(sys.getsizeof(ob.__dict__))

56
112


因此实际的内存占用是56+112即168,也就是每1000000个数据会占用168m的内存

### `*`dataclass

标准库dataclass提供了另一种快速定义类的方法,它本质上和直接定义类一样

In [12]:
from dataclasses import dataclass
@dataclass
class Point:
    x: int
    y: int
    z: int
        
ob = Point(1,2,3)
print(sys.getsizeof(ob))
print(sys.getsizeof(ob.__dict__))

56
112


它还可以定义不可变类型,不过不可变类型性能会差些,但它依然是一般的类实例

In [13]:
from dataclasses import dataclass
@dataclass(frozen=True)
class Point:
    x: int
    y: int
    z: int
        
ob = Point(1,2,3)
print(sys.getsizeof(ob))
print(sys.getsizeof(ob.__dict__))

56
112


## 用`__slots__`限定的类定义结构化数据

使用`__slots__`限定的类可以去掉`__weakref__`和`__dict__`字段,因此内存占用可以进一步减小.以上面的例子为例,其结构大致为:


字段|大小(bit)
---|---
`PyGC_Head`|24
`PyObject_Head`|16
`x`|8
`y`|8
`z`|8



In [14]:
class Point:
    __slots__= "x","y","z"
    def __init__(self,x,y,z):
        self.x = x
        self.y = y
        self.z = z

ob = Point(1,2,3)
print(sys.getsizeof(ob))

64


也就是每1000000个数据会占用64m的内存

## 具名元组

`collections.namedtuple`是python中的具名元组,它的结构更加紧凑,他们是不可变对象.由于内存中元组还包含字段信息,因此还是会比使用`__slots__`的类内存占用多些.

In [15]:
from collections import namedtuple

In [17]:
Point = namedtuple(
    'Point', 
    [
        'x',
        'y',
        'z'
    ]
)
ob = Point(1,2,3)
print(sys.getsizeof(ob))

72


也就是每1000000个数据会占用72m的内存
|

## 更进一步

上面可以看出使用python及标准库我们内存占用最小的结构化数据对象就是用__slots__限定的类,如果我们希望可以进一步压缩空间,可以使用:


+ numpy的结构体
+ cython的类

### numpy的结构体

"准标准库"numpy提供了定义结构体的能力,可以使用接口`dtype()`来定义结构紧凑的结构体,需要注意的是这个接口必须申明数据类型(c语言对应的,具体可以看[这里]()),numpy的结构体只能用于numpy的数据结构中,因此使用有一定局限性,但如果是处理大规模数据,那就完全不是问题

In [27]:
import numpy as np

Point = np.dtype(
    {'names':['x','y','z'],'formats':['i','i','i']
},align=True)
ob = np.zeros(1000000,dtype=Point)
print(sys.getsizeof(ob))

12000096


也就是每1000000个数据会占用12m的内存

### cython定义类



In [1]:
%load_ext Cython

In [2]:
%%cython
import sys
import cython

@cython.cclass
class Point:
    x: cython.int
    y: cython.int
    z: cython.int
    def __init__(self, x,y,z):
        self.x = x
        self.y = y
        self.z = z

        
ob = Point(1,2,3)
print(sys.getsizeof(ob))

LinkError: command 'C:\\Program Files\\mingw-w64\\x86_64-8.1.0-posix-seh-rt_v6-rev0\\mingw64\\bin\\gcc.exe' failed with exit status 1

使用cython定义需要注意,每次调用字段都会做一次c语言类型到python类型的转换